
# Car Resale Value Calclulator

### This notebook serves the purposee of cleaning the raw data The regression fitting and graphing willl be done in subsequent notebooks

#### First we import all neccesary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
import plotly.express as px
warnings.filterwarnings('ignore')

#### Now we read in the dataframe

In [ ]:
df = pd.read_csv('../input/used-cars-dataset-from-cardekhocom/Cardekho_Extract.csv')
df.head()

In [ ]:
df.info()

#### dropping unwanted columns

In [ ]:
df.drop('Source.Name', inplace=True, axis=1)
df.drop('web-scraper-start-url', inplace=True, axis=1)
df.drop('web-scraper-order', inplace=True, axis=1)
df.drop('mileage', inplace=True, axis=1)
df.drop('new-price', inplace=True, axis=1)
df.drop('max_power', inplace=True, axis=1)
df.head()

#### Split the Name into just the model and the maker and capitalsing them

In [ ]:
tem = df["full_name"].str.upper()
df['full_name'] = tem
new = df["full_name"].str.split(" ", n = 2, expand = True)
  
# making separate first name column from new data frame
df["Maker"]= new[0]
df['Model']= new[1]
df.drop('full_name', inplace=True, axis=1)
df.head()

#### removing aplhabets from alphanumeric strings

In [ ]:
new = df["selling_price"].str.split(" ", n = 1, expand = True)
df["selling_price"]= new[0]

new = df["km_driven"].str.split(" ", n = 1, expand = True)
df["km_driven"]= new[0]

new = df["engine"].str.split("ne", n = 1, expand = True)
df["engine"]= new[1]
new = df["engine"].str.split(" ", n = 1, expand = True)
df["engine"]= new[0]

df.head()

#### Converting year of purchase to age of vehicle

In [ ]:
age = []
age2 = []
#age3 = []
for m in df['year']:
    age.append(2021-m)
    age2.append((2021-m)**2)
    #age3.append((2021-m)**3)
df['Age'] = age
df['Age2']= age2
#df['Age3']= age3
df.drop('year', inplace=True, axis=1)

#### Converting transmisission to boolean

In [ ]:
trans = pd.get_dummies(df['transmission_type'])
df = pd.concat([df,trans],axis=1)
df.drop('transmission_type', inplace=True, axis=1)

#### Converting fuel to boolean

In [ ]:
fuel = pd.get_dummies(df['fuel_type'])
df = pd.concat([df,fuel],axis=1)
df.drop('fuel_type', inplace=True, axis=1)

#### Converting seller type to boolean

In [ ]:
seller = pd.get_dummies(df['seller_type'])
df = pd.concat([df,seller],axis=1)
df.drop('seller_type', inplace=True, axis=1)

#### Convert Owner Type to boolean

In [ ]:
owner = pd.get_dummies(df['owner_type'])
df = pd.concat([df,owner],axis=1)
df.drop('owner_type', inplace=True, axis=1)

#### Convert Seat Type to boolean

In [ ]:
seats = pd.get_dummies(df['seats'])
df = pd.concat([df,seats],axis=1)
df.drop('seats', inplace=True, axis=1)

#### Converting sell price to numbers

In [ ]:
#kms driven
df['selling_price'] = df['selling_price'].astype(str)
SP = []
for m in df['selling_price']:
    if "*" in m :
        m =float(m.replace(',', '').replace('*', ''))/100000
    SP.append(float(m))
df['selling_price'] = SP

#### Converting kms,engine,max power to numbers

In [ ]:
#kms driven
df['km_driven'] = df['km_driven'].astype(str)
KM = []
for m in df['km_driven']:
    KM.append(m.replace(',', ''))
df['km_driven'] = KM
df = df.dropna()
df["km_driven"] = pd.to_numeric(df["km_driven"], downcast="float")
km2 = []
#km3 = []
km = []
for m in df['km_driven']:
    km.append(m/10000)
    km2.append((m**2)/100000000)
    #km3.append((m**3)/1000000000000)
df['km_driven2'] = km2
df['km_driven'] = km
#df['km_driven3'] = km3

#engine size
df["engine"] = pd.to_numeric(df["engine"], downcast="float")

#df

#### Now checking the dataframe

In [ ]:
df.info()

### Taking User input

In [ ]:
maker = input("Vehicle manafacturer:").upper()
model = input("Vehicle model:").upper()

### Creating a feature and target dataframe

In [ ]:
data = df

temp = data[data['Model']==model]
nan_value = float("NaN")
temp.replace(0, nan_value, inplace=True)
temp.dropna(how='all', axis=1, inplace=True)
temp.replace(nan_value, 0, inplace=True)
temp.drop('Maker', inplace=True, axis=1)
temp.drop('Model', inplace=True, axis=1)

#temp.drop('Age2', inplace=True, axis=1)
#temp.drop('km_driven2', inplace=True, axis=1)

#temp.drop('Age3', inplace=True, axis=1)
#temp.drop('km_driven3', inplace=True, axis=1)
temp.info()


In [ ]:
Y = temp['selling_price']

X = list(temp)
X.remove('selling_price')
X = temp[X]

### Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X.fillna(0))

### Plotting the data

In [ ]:
plt.figure(figsize = (12,6))
sns.heatmap(temp.corr(method = 'spearman'),annot=True, cmap='Greys')
#temp.corr(method = 'spearman')

Here we see that the strongest correlation (negative) is between selling price and kilometrers done by the car, the age of the car and the transmisssion type (the selling price is drastically less for a manual car)

Lets us graphically see the realtion between these 2 parameters with the selling price

In [ ]:
sns.lmplot(x='Age',y='selling_price',data=temp,aspect=01.2,size=5)

In [ ]:
sns.lmplot(x= 'km_driven',y='selling_price',data=temp,aspect=1.2,size=5)

In [ ]:
plt.figure(figsize = (10,13))
fig =px.scatter(temp, x ='Age',y='selling_price',color='km_driven',width=1000, height=600 )
fig.update_layout(legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1))
fig.show()

### Splitting the data into features and targets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=52)

### Creating and fitting the model

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train,Y_train)

### Checking coeffs

In [ ]:
coeff_df = pd.DataFrame(lr.coef_,X.columns,columns=['Coefficient'])
intercept = lr.intercept_
print(coeff_df,"\nIntecept =",intercept)

### Checking CV score
With respect to training data

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr,X_train,Y_train,cv=5,scoring='r2')
    
print('CV Mean: ', np.mean(scores))
print('STD: ', np.std(scores))

### Testing the model

In [ ]:
h = lr.predict(X_test)
plt.scatter(Y_test,h)

Error distribution

In [ ]:
sns.distplot((Y_test-h),bins=150,axlabel = "error")

### Evaluating the model

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(Y_test, h))
print('MSE:', metrics.mean_squared_error(Y_test, h))
print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, h)))
print('Model Accuracy:',lr.score(X_test, Y_test)*100,'%')